In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-employee-attrition/sample_submission_znWiLZ4.csv
/kaggle/input/predicting-employee-attrition/test_hXY9mYw.csv
/kaggle/input/predicting-employee-attrition/train_MpHjUjU.csv


In [2]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score,precision_recall_curve
from sklearn.preprocessing import LabelEncoder

In [3]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [4]:
train = pd.read_csv('/kaggle/input/predicting-employee-attrition/train_MpHjUjU.csv')
test = pd.read_csv('/kaggle/input/predicting-employee-attrition/test_hXY9mYw.csv')
subm = pd.read_csv('/kaggle/input/predicting-employee-attrition/sample_submission_znWiLZ4.csv')

In [5]:
print(test.shape)

(741, 1)


In [6]:
print(train.shape)
print(train.columns)
train.head(25)

(19104, 13)
Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating'],
      dtype='object')


,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1
7,2017-02-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1
8,2017-03-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,350000,1
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1


In [7]:
print(train['Education_Level'].nunique())
train['City'].nunique()

3


29

**Preprocessing**

In [8]:
target='is_resigned'

In [9]:
def create_target_binary(data):
    data[target] = ~data['LastWorkingDate'].isnull()
    data[target] = data[target].astype(int)

In [10]:
create_target_binary(train)
print(train[train[target]==1].shape[0])
train[target].head(10)

1616


0    0
1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    0
9    1
Name: is_resigned, dtype: int64

In [11]:
#convert fields to date format
def preproc_date_fields(data):
    format= '%Y-%m-%d'
    data['MMM-YY'] = pd.to_datetime(data['MMM-YY'],format=format)
    data['Dateofjoining'] = pd.to_datetime(data['Dateofjoining'],format=format)
    data['LastWorkingDate'] = pd.to_datetime(data['LastWorkingDate'],format=format)
    
    data= gen_date_features(data)
    
def gen_date_features(data):
    #compute number of months since joined as on reporting date
    data['months_since_joined'] =((data['MMM-YY'] - data['Dateofjoining'])/np.timedelta64(1, 'M')).astype(int)
    data['days_since_joined'] =((data['MMM-YY'] - data['Dateofjoining'])/np.timedelta64(1, 'D')).astype(int)


In [12]:
preproc_date_fields(train)

In [13]:
print(train.columns)
train.head(10)

Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating',
       'is_resigned', 'months_since_joined', 'days_since_joined'],
      dtype='object')


,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,is_resigned,months_since_joined,days_since_joined
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,0,0,8
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,0,1,39
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,1,2,68
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,0,0,-5
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,0,0,25
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1,0,0,-6
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1,0,0,25
7,2017-02-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1,0,1,56
8,2017-03-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,350000,1,0,2,84
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1,1,3,115


In [14]:
def target_6months_fill_fn(data):
    max_months_since_joined = data['months_since_joined'].max()
    max_is_resigned = data['is_resigned'].max()
    temp = data['is_resigned'].copy()
    temp[data['months_since_joined'] > (max_months_since_joined-6)]=max_is_resigned
    return temp

# def target_6months_fill_fn(data,max_months_since_joined,max_is_resigned):
#     temp = data['is_resigned'].copy()
#     temp[data['months_since_joined'] >= (max_months_since_joined-6)]=max_is_resigned
#     return temp

In [15]:
%%time
#populate target to 1 if employee resigned in next 6 months
#(i.e) target of 1 is backward filled for previous 5 months also
result = train.groupby('Emp_ID').apply(target_6months_fill_fn)
result=result.reset_index(level=[0,1])['is_resigned']
train['is_resigned'] = result

keycols = ['MMM-YY','Emp_ID','is_resigned','days_since_joined',
           'Dateofjoining','LastWorkingDate','months_since_joined',
           ]
train[keycols].head(100)
# x = train.groupby('Emp_ID').transform(lambda gp:target_6months_fill_fn(gp,gp[]))

CPU times: user 3.54 s, sys: 20.2 ms, total: 3.56 s
Wall time: 3.55 s


,MMM-YY,Emp_ID,is_resigned,days_since_joined,Dateofjoining,LastWorkingDate,months_since_joined
0,2016-01-01,1,1,8,2015-12-24,NaT,0
1,2016-02-01,1,1,39,2015-12-24,NaT,1
2,2016-03-01,1,1,68,2015-12-24,2016-03-11,2
3,2017-11-01,2,0,-5,2017-11-06,NaT,0
4,2017-12-01,2,0,25,2017-11-06,NaT,0
5,2016-12-01,4,1,-6,2016-12-07,NaT,0
6,2017-01-01,4,1,25,2016-12-07,NaT,0
7,2017-02-01,4,1,56,2016-12-07,NaT,1
8,2017-03-01,4,1,84,2016-12-07,NaT,2
9,2017-04-01,4,1,115,2016-12-07,2017-04-27,3


**Generate lag features**

In [16]:
print(train.columns)

Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating',
       'is_resigned', 'months_since_joined', 'days_since_joined'],
      dtype='object')


In [17]:
from statsmodels.tsa.tsatools import lagmat

Create Test Data

In [18]:
test['MMM-YY'] = pd.to_datetime('2018-01-01')

In [19]:
# mask1 = train['Emp_ID'].isin(test['Emp_ID'])
cols = [   'Gender', 'City', 'Education_Level',
        'Dateofjoining',  'Joining Designation' ]
train_emp_unchanged = train.groupby('Emp_ID')[cols].first()
train_emp_unchanged.head(25)

,Gender,City,Education_Level,Dateofjoining,Joining Designation
Emp_ID,,,,,
1,Male,C23,Master,2015-12-24,1
2,Male,C7,Master,2017-11-06,2
4,Male,C13,Master,2016-12-07,2
5,Male,C9,College,2016-01-09,1
6,Female,C11,Bachelor,2017-07-31,3
8,Male,C2,College,2017-09-19,3
11,Female,C19,Master,2017-12-07,1
12,Male,C23,Master,2016-06-29,1
13,Male,C19,Master,2012-05-28,1


In [20]:
test = test.join(train_emp_unchanged,on='Emp_ID')
test.sort_values('Emp_ID',inplace=True)
print(test.shape)
test.head(25)

(741, 7)


,Emp_ID,MMM-YY,Gender,City,Education_Level,Dateofjoining,Joining Designation
619,2,2018-01-01,Male,C7,Master,2017-11-06,2
429,6,2018-01-01,Female,C11,Bachelor,2017-07-31,3
670,11,2018-01-01,Female,C19,Master,2017-12-07,1
588,14,2018-01-01,Female,C26,College,2017-10-16,3
120,25,2018-01-01,Male,C24,Bachelor,2014-10-30,1
140,26,2018-01-01,Male,C14,Master,2015-05-07,1
392,31,2018-01-01,Female,C28,Master,2017-07-04,3
467,39,2018-01-01,Male,C26,College,2017-08-08,2
214,41,2018-01-01,Male,C29,Master,2016-07-04,4
361,45,2018-01-01,Male,C8,Master,2017-05-29,3


In [21]:
gen_date_features(test)
print(test.shape)
test.head(25)

(741, 9)


,Emp_ID,MMM-YY,Gender,City,Education_Level,Dateofjoining,Joining Designation,months_since_joined,days_since_joined
619,2,2018-01-01,Male,C7,Master,2017-11-06,2,1,56
429,6,2018-01-01,Female,C11,Bachelor,2017-07-31,3,5,154
670,11,2018-01-01,Female,C19,Master,2017-12-07,1,0,25
588,14,2018-01-01,Female,C26,College,2017-10-16,3,2,77
120,25,2018-01-01,Male,C24,Bachelor,2014-10-30,1,38,1159
140,26,2018-01-01,Male,C14,Master,2015-05-07,1,31,970
392,31,2018-01-01,Female,C28,Master,2017-07-04,3,5,181
467,39,2018-01-01,Male,C26,College,2017-08-08,2,4,146
214,41,2018-01-01,Male,C29,Master,2016-07-04,4,17,546
361,45,2018-01-01,Male,C8,Master,2017-05-29,3,7,217


Combine train and test to create lag columns

In [22]:
train['istrain'] = 1
test['istrain'] = 0

combined = pd.concat([train,test],axis=0)
combined.sort_values(['Emp_ID','MMM-YY'],inplace=True)
print(combined.shape)
combined.head()

(19845, 17)


,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,is_resigned,months_since_joined,days_since_joined,istrain
0,2016-01-01,1,28.0,Male,C23,Master,57387.0,2015-12-24,NaT,1,1.0,2381060.0,2.0,1.0,0,8,1
1,2016-02-01,1,28.0,Male,C23,Master,57387.0,2015-12-24,NaT,1,1.0,-665480.0,2.0,1.0,1,39,1
2,2016-03-01,1,28.0,Male,C23,Master,57387.0,2015-12-24,2016-03-11,1,1.0,0.0,2.0,1.0,2,68,1
3,2017-11-01,2,31.0,Male,C7,Master,67016.0,2017-11-06,NaT,2,2.0,0.0,1.0,0.0,0,-5,1
4,2017-12-01,2,31.0,Male,C7,Master,67016.0,2017-11-06,NaT,2,2.0,0.0,1.0,0.0,0,25,1


In [23]:
# #remove lag cols
# laggedcols = [col for col in train.columns if '_lag_' in col]
# train.drop(laggedcols,axis=1,inplace=True)
# citycols = [col for col in train.columns if 'city_' in col]
# train.drop(citycols,axis=1,inplace=True)

In [24]:
def generate_lag_features(data,cols_to_lag,groupcol,shift_range):
    lagged = pd.DataFrame()
    for i,shift in enumerate(shift_range):
        shifted = data[cols_to_lag+[groupcol]].groupby(groupcol).shift(shift)
        cur_lagged = shifted.rename(columns=lambda x: x+"_lag_"+str(shift))
        if i==0:
            lagged = cur_lagged
        else:
            lagged = pd.concat([lagged, cur_lagged],axis=1)
#             lagged = lagged.join(cur_lagged)
    return lagged

In [25]:
cols_to_lag =['Age','Total Business Value','Quarterly Rating',
             'Salary','Designation']
groupcol='Emp_ID'
shift_range=[1,2,3]
lagged = generate_lag_features(combined,cols_to_lag,groupcol,shift_range)
# combined = combined.join(lagged)
print(lagged.shape)
combined = pd.concat([combined, lagged],axis=1)
print(combined.columns)
laggedcols = [col for col in combined.columns if '_lag_' in col]
keycols = ['MMM-YY','Emp_ID','Total Business Value',
           'Quarterly Rating',
           'is_resigned','Dateofjoining','LastWorkingDate','months_since_joined',
           ]
print(combined.shape)
combined[laggedcols+keycols].head(25)

(19845, 15)
Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating',
       'is_resigned', 'months_since_joined', 'days_since_joined', 'istrain',
       'Age_lag_1', 'Total Business Value_lag_1', 'Quarterly Rating_lag_1',
       'Salary_lag_1', 'Designation_lag_1', 'Age_lag_2',
       'Total Business Value_lag_2', 'Quarterly Rating_lag_2', 'Salary_lag_2',
       'Designation_lag_2', 'Age_lag_3', 'Total Business Value_lag_3',
       'Quarterly Rating_lag_3', 'Salary_lag_3', 'Designation_lag_3'],
      dtype='object')
(19845, 32)


,Age_lag_1,Total Business Value_lag_1,Quarterly Rating_lag_1,Salary_lag_1,Designation_lag_1,Age_lag_2,Total Business Value_lag_2,Quarterly Rating_lag_2,Salary_lag_2,Designation_lag_2,Age_lag_3,Total Business Value_lag_3,Quarterly Rating_lag_3,Salary_lag_3,Designation_lag_3,MMM-YY,Emp_ID,Total Business Value,Quarterly Rating,is_resigned,Dateofjoining,LastWorkingDate,months_since_joined
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01,1,2381060.0,2.0,1.0,2015-12-24,NaT,0
1,28.0,2381060.0,2.0,57387.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-01,1,-665480.0,2.0,1.0,2015-12-24,NaT,1
2,28.0,-665480.0,2.0,57387.0,1.0,28.0,2381060.0,2.0,57387.0,1.0,NaN,NaN,NaN,NaN,NaN,2016-03-01,1,0.0,2.0,1.0,2015-12-24,2016-03-11,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-01,2,0.0,1.0,0.0,2017-11-06,NaT,0
4,31.0,0.0,1.0,67016.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-01,2,0.0,1.0,0.0,2017-11-06,NaT,0
619,31.0,0.0,1.0,67016.0,2.0,31.0,0.0,1.0,67016.0,2.0,NaN,NaN,NaN,NaN,NaN,2018-01-01,2,NaN,NaN,NaN,2017-11-06,NaT,1
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-01,4,0.0,1.0,1.0,2016-12-07,NaT,0
6,43.0,0.0,1.0,65603.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01,4,0.0,1.0,1.0,2016-12-07,NaT,0
7,43.0,0.0,1.0,65603.0,2.0,43.0,0.0,1.0,65603.0,2.0,NaN,NaN,NaN,NaN,NaN,2017-02-01,4,0.0,1.0,1.0,2016-12-07,NaT,1
8,43.0,0.0,1.0,65603.0,2.0,43.0,0.0,1.0,65603.0,2.0,43.0,0.0,1.0,65603.0,2.0,2017-03-01,4,350000.0,1.0,1.0,2016-12-07,NaT,2


In [26]:
def gen_simple_expsm(df):
    temp = pd.DataFrame()
    for i,alpha in enumerate(alpha_list):
        temp['Total Business Value_expsm'+str(i)] = df['Total Business Value'].ewm(alpha=alpha).mean()
    return temp

In [27]:
alpha_list=[0.1,0.4,0.7]
result = combined.groupby('Emp_ID').apply(gen_simple_expsm)
combined = pd.concat([combined,result],axis=1)
print(combined.shape)
result.head()

(19845, 35)


,Total Business Value_expsm0,Total Business Value_expsm1,Total Business Value_expsm2
0,2.381060e+06,2.381060e+06,2.381060e+06
1,7.776179e+05,4.769725e+05,3.756769e+04
2,4.906740e+05,2.336192e+05,1.054058e+04
3,0.000000e+00,0.000000e+00,0.000000e+00
4,0.000000e+00,0.000000e+00,0.000000e+00


In [28]:
# lag = 1
# lagged = lag_func(train,lag,'Total Business Value')
# print(lagged.columns)
# lagged.head(25)
# # last_date = lagged['date'].max()

In [29]:
#Label encoding of categorical variables
catcols = ['Gender','Education_Level']
le = LabelEncoder()

for col in catcols:
    le.fit(combined[col])
    print(le.classes_)
    combined[col]=le.transform(combined[col])
    
combined[catcols].head()

['Female' 'Male']
['Bachelor' 'College' 'Master']


,Gender,Education_Level
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2


In [30]:
#One hot encoding of categorical variables
catcols = ['City']
combined = combined.join(pd.get_dummies(train[catcols], prefix='city'))
print(combined.columns)
combined.head()

Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating',
       'is_resigned', 'months_since_joined', 'days_since_joined', 'istrain',
       'Age_lag_1', 'Total Business Value_lag_1', 'Quarterly Rating_lag_1',
       'Salary_lag_1', 'Designation_lag_1', 'Age_lag_2',
       'Total Business Value_lag_2', 'Quarterly Rating_lag_2', 'Salary_lag_2',
       'Designation_lag_2', 'Age_lag_3', 'Total Business Value_lag_3',
       'Quarterly Rating_lag_3', 'Salary_lag_3', 'Designation_lag_3',
       'Total Business Value_expsm0', 'Total Business Value_expsm1',
       'Total Business Value_expsm2', 'city_C1', 'city_C10', 'city_C11',
       'city_C12', 'city_C13', 'city_C14', 'city_C15', 'city_C16', 'city_C17',
       'city_C18', 'city_C19', 'city_C2', 'city_C20', 'city_C21', 'city_C22',
       'city_C23', 'city_C24', 'city_C25', 'city_C26', 'ci

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,is_resigned,months_since_joined,days_since_joined,istrain,Age_lag_1,Total Business Value_lag_1,Quarterly Rating_lag_1,Salary_lag_1,Designation_lag_1,Age_lag_2,Total Business Value_lag_2,Quarterly Rating_lag_2,Salary_lag_2,Designation_lag_2,Age_lag_3,Total Business Value_lag_3,Quarterly Rating_lag_3,Salary_lag_3,Designation_lag_3,Total Business Value_expsm0,Total Business Value_expsm1,Total Business Value_expsm2,city_C1,city_C10,city_C11,city_C12,city_C13,city_C14,city_C15,city_C16,city_C17,city_C18,city_C19,city_C2,city_C20,city_C21,city_C22,city_C23,city_C24,city_C25,city_C26,city_C27,city_C28,city_C29,city_C3,city_C4,city_C5,city_C6,city_C7,city_C8,city_C9
0,2016-01-01,1,28.0,1,C23,2,57387.0,2015-12-24,NaT,1,1.0,2381060.0,2.0,1.0,0,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.381060e+06,2.381060e+06,2.381060e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2018-01-01,394,NaN,0,C20,2,NaN,2011-06-06,NaT,2,NaN,NaN,NaN,NaN,78,2401,0,34.0,2701750.0,3.0,97722.0,4.0,34.0,1433660.0,3.0,97722.0,4.0,34.0,1758980.0,3.0,97722.0,4.0,1.274849e+06,1.827347e+06,2.322109e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2016-02-01,1,28.0,1,C23,2,57387.0,2015-12-24,NaT,1,1.0,-665480.0,2.0,1.0,1,39,1,28.0,2381060.0,2.0,57387.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.776179e+05,4.769725e+05,3.756769e+04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,173,NaN,1,C28,1,NaN,2011-06-13,NaT,1,NaN,NaN,NaN,NaN,78,2394,0,39.0,706010.0,3.0,56174.0,3.0,39.0,1102120.0,3.0,56174.0,3.0,39.0,475020.0,3.0,56174.0,3.0,7.871552e+05,7.376495e+05,7.695143e+05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016-03-01,1,28.0,1,C23,2,57387.0,2015-12-24,2016-03-11,1,1.0,0.0,2.0,1.0,2,68,1,28.0,-665480.0,2.0,57387.0,1.0,28.0,2381060.0,2.0,57387.0,1.0,NaN,NaN,NaN,NaN,NaN,4.906740e+05,2.336192e+05,1.054058e+04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


Split Train and Test

In [31]:
train = combined[combined['istrain']==1]
test = combined[combined['istrain']==0]

In [32]:
print(train.shape)
print(test.shape)

(19104, 64)
(741, 64)


In [33]:
del test['is_resigned']

Split Train and Validation

In [34]:
#6 months period used for validation is 2017-07-01 till 2017-12-01
train_mask = (train['MMM-YY'].dt.year!=2017) | (train['MMM-YY'].dt.month<7)
train_model = train[train_mask].dropna(subset=['Age_lag_1'])
#retain only one record for each employee with start of 6 months period
valid_mask = (train['MMM-YY'].dt.year==2017) & (train['MMM-YY'].dt.month==7)
valid_model = train[valid_mask].dropna(subset=['Age_lag_1'])
print(valid_model.shape)
print(train_model.shape)

(706, 64)
(12311, 64)


In [35]:
valid_model.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,is_resigned,months_since_joined,days_since_joined,istrain,Age_lag_1,Total Business Value_lag_1,Quarterly Rating_lag_1,Salary_lag_1,Designation_lag_1,Age_lag_2,Total Business Value_lag_2,Quarterly Rating_lag_2,Salary_lag_2,Designation_lag_2,Age_lag_3,Total Business Value_lag_3,Quarterly Rating_lag_3,Salary_lag_3,Designation_lag_3,Total Business Value_expsm0,Total Business Value_expsm1,Total Business Value_expsm2,city_C1,city_C10,city_C11,city_C12,city_C13,city_C14,city_C15,city_C16,city_C17,city_C18,city_C19,city_C2,city_C20,city_C21,city_C22,city_C23,city_C24,city_C25,city_C26,city_C27,city_C28,city_C29,city_C3,city_C4,city_C5,city_C6,city_C7,city_C8,city_C9
46,2017-07-01,13,30.0,1,C19,2,119227.0,2012-05-28,NaT,1,4.0,150000.0,1.0,1.0,61,1860,1,30.0,258610.0,1.0,119227.0,4.0,30.0,400000.0,1.0,119227.0,4.0,30.0,0.0,1.0,119227.0,4.0,4.160331e+05,2.807670e+05,194022.143726,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
132,2017-07-01,25,30.0,1,C24,0,102077.0,2014-10-30,NaT,1,3.0,650000.0,4.0,0.0,32,975,1,30.0,1219340.0,3.0,102077.0,3.0,30.0,350000.0,3.0,102077.0,3.0,30.0,-414250.0,3.0,102077.0,3.0,1.314819e+06,8.874466e+05,748911.892455,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
156,2017-07-01,26,42.0,1,C14,2,132577.0,2015-05-07,NaT,1,4.0,153590.0,2.0,0.0,25,786,1,42.0,0.0,2.0,132577.0,4.0,42.0,970030.0,2.0,132577.0,4.0,42.0,1940050.0,2.0,132577.0,4.0,2.888970e+06,1.056600e+06,261463.413404,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
195,2017-07-01,36,41.0,0,C18,0,32865.0,2016-07-01,NaT,2,2.0,984330.0,2.0,1.0,11,365,1,41.0,139080.0,3.0,32865.0,2.0,40.0,0.0,3.0,32865.0,2.0,40.0,1460000.0,3.0,32865.0,2.0,4.033730e+05,5.704047e+05,746264.487621,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
199,2017-07-01,37,33.0,0,C14,0,57375.0,2017-05-16,NaT,2,2.0,350020.0,1.0,1.0,1,46,1,33.0,300000.0,1.0,57375.0,2.0,33.0,0.0,1.0,57375.0,2.0,NaN,NaN,NaN,NaN,NaN,2.287897e+05,2.704184e+05,316561.151079,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
train_model.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,is_resigned,months_since_joined,days_since_joined,istrain,Age_lag_1,Total Business Value_lag_1,Quarterly Rating_lag_1,Salary_lag_1,Designation_lag_1,Age_lag_2,Total Business Value_lag_2,Quarterly Rating_lag_2,Salary_lag_2,Designation_lag_2,Age_lag_3,Total Business Value_lag_3,Quarterly Rating_lag_3,Salary_lag_3,Designation_lag_3,Total Business Value_expsm0,Total Business Value_expsm1,Total Business Value_expsm2,city_C1,city_C10,city_C11,city_C12,city_C13,city_C14,city_C15,city_C16,city_C17,city_C18,city_C19,city_C2,city_C20,city_C21,city_C22,city_C23,city_C24,city_C25,city_C26,city_C27,city_C28,city_C29,city_C3,city_C4,city_C5,city_C6,city_C7,city_C8,city_C9
1,2016-02-01,1,28.0,1,C23,2,57387.0,2015-12-24,NaT,1,1.0,-665480.0,2.0,1.0,1,39,1,28.0,2381060.0,2.0,57387.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,777617.894737,476972.500000,37567.692308,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016-03-01,1,28.0,1,C23,2,57387.0,2015-12-24,2016-03-11,1,1.0,0.0,2.0,1.0,2,68,1,28.0,-665480.0,2.0,57387.0,1.0,28.0,2381060.0,2.0,57387.0,1.0,NaN,NaN,NaN,NaN,NaN,490674.022140,233619.183673,10540.575540,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2017-01-01,4,43.0,1,C13,2,65603.0,2016-12-07,NaT,2,2.0,0.0,1.0,1.0,0,25,1,43.0,0.0,1.0,65603.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2017-02-01,4,43.0,1,C13,2,65603.0,2016-12-07,NaT,2,2.0,0.0,1.0,1.0,1,56,1,43.0,0.0,1.0,65603.0,2.0,43.0,0.0,1.0,65603.0,2.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2017-03-01,4,43.0,1,C13,2,65603.0,2016-12-07,NaT,2,2.0,350000.0,1.0,1.0,2,84,1,43.0,0.0,1.0,65603.0,2.0,43.0,0.0,1.0,65603.0,2.0,43.0,0.0,1.0,65603.0,2.0,101773.771445,160845.588235,247000.705716,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Model**

In [37]:
#custom F1 metric
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
#     y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
#     return 'f1', f1_score(y_true, y_hat), True    
    opt_cutoff, f1score = get_opt_cutoff_prec(y_true,y_hat)
    return 'F1-cutoff', f1score, True

In [38]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

#This function converts the predicted probabilities into labels using specified cutoff
def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [39]:
train.columns

Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating',
       'is_resigned', 'months_since_joined', 'days_since_joined', 'istrain',
       'Age_lag_1', 'Total Business Value_lag_1', 'Quarterly Rating_lag_1',
       'Salary_lag_1', 'Designation_lag_1', 'Age_lag_2',
       'Total Business Value_lag_2', 'Quarterly Rating_lag_2', 'Salary_lag_2',
       'Designation_lag_2', 'Age_lag_3', 'Total Business Value_lag_3',
       'Quarterly Rating_lag_3', 'Salary_lag_3', 'Designation_lag_3',
       'Total Business Value_expsm0', 'Total Business Value_expsm1',
       'Total Business Value_expsm2', 'city_C1', 'city_C10', 'city_C11',
       'city_C12', 'city_C13', 'city_C14', 'city_C15', 'city_C16', 'city_C17',
       'city_C18', 'city_C19', 'city_C2', 'city_C20', 'city_C21', 'city_C22',
       'city_C23', 'city_C24', 'city_C25', 'city_C26', 'ci

In [40]:
#features for training model
features = ['Gender', 'Education_Level','Joining Designation',
            'months_since_joined',
#             'Total Business Value_expsm0','Total Business Value_expsm1','Total Business Value_expsm2'
#             'days_since_joined'
           ]
lagcols = [col for col in train.columns if '_lag_' in col]
citycols = [col for col in train.columns if 'city_' in col]
features += lagcols + citycols
print(len(features))
print(features)

48
['Gender', 'Education_Level', 'Joining Designation', 'months_since_joined', 'Age_lag_1', 'Total Business Value_lag_1', 'Quarterly Rating_lag_1', 'Salary_lag_1', 'Designation_lag_1', 'Age_lag_2', 'Total Business Value_lag_2', 'Quarterly Rating_lag_2', 'Salary_lag_2', 'Designation_lag_2', 'Age_lag_3', 'Total Business Value_lag_3', 'Quarterly Rating_lag_3', 'Salary_lag_3', 'Designation_lag_3', 'city_C1', 'city_C10', 'city_C11', 'city_C12', 'city_C13', 'city_C14', 'city_C15', 'city_C16', 'city_C17', 'city_C18', 'city_C19', 'city_C2', 'city_C20', 'city_C21', 'city_C22', 'city_C23', 'city_C24', 'city_C25', 'city_C26', 'city_C27', 'city_C28', 'city_C29', 'city_C3', 'city_C4', 'city_C5', 'city_C6', 'city_C7', 'city_C8', 'city_C9']


In [41]:
#Light GBM model
y_train = train_model[target]
y_val = valid_model[target]

x_train = train_model[features]
x_val = valid_model[features]
designation_cols = [col for col in train.columns if 'Designation_lag_' in col]
categoricals = designation_cols + ['Gender', 'Education_Level', 'Joining Designation']
train_set = lgb.Dataset(x_train,y_train , 
#                                 categorical_feature=categoricals
                       )
val_set = lgb.Dataset(x_val, y_val, 
#                               categorical_feature=categoricals
                     )


In [42]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'AUC',
    'subsample': 0.75,
    'subsample_freq': 30,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
     'max_depth': 15,
    'num_leaves': 60,
#     'is_unbalance': True,
    'lambda_l1': 0.3,  
    'lambda_l2': 0.7,
    'verbose': -1,
#             'eval_metric': 'cappa'
    }

In [43]:
model = lgb.train(params, train_set, num_boost_round = 10000, 
                  early_stopping_rounds = 100, 
                  keep_training_booster=True,
                  feval = lgb_f1_score,
                  valid_sets=[train_set, val_set], 
                  verbose_eval = 50,
                 )

Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.899402	training's F1-cutoff: 0.795081	valid_1's auc: 0.80489	valid_1's F1-cutoff: 0.711965
[100]	training's auc: 0.940143	training's F1-cutoff: 0.839915	valid_1's auc: 0.820655	valid_1's F1-cutoff: 0.723944
[150]	training's auc: 0.963161	training's F1-cutoff: 0.876095	valid_1's auc: 0.836153	valid_1's F1-cutoff: 0.741463
[200]	training's auc: 0.976775	training's F1-cutoff: 0.905468	valid_1's auc: 0.844354	valid_1's F1-cutoff: 0.7584
[250]	training's auc: 0.985165	training's F1-cutoff: 0.925226	valid_1's auc: 0.849194	valid_1's F1-cutoff: 0.768987
[300]	training's auc: 0.990825	training's F1-cutoff: 0.941267	valid_1's auc: 0.854552	valid_1's F1-cutoff: 0.764992
[350]	training's auc: 0.994019	training's F1-cutoff: 0.955394	valid_1's auc: 0.855916	valid_1's F1-cutoff: 0.764228
[400]	training's auc: 0.996088	training's F1-cutoff: 0.965887	valid_1's auc: 0.857609	valid_1's F1-cutoff: 0.76874
Early stopping,

In [44]:
#Predictions
#predict validation set
valid_iteration = model.best_iteration

In [45]:
val_preds =  model.predict(x_val, num_iteration=valid_iteration)

#determine validation probability threshold at which F1 score is optimized
opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_preds)
val_labels = convert_probtolabels(val_preds,cutoff=opt_cutoff)        

valid_score = f1_score(y_val, val_labels,average='macro')
print("Valid F1 score: {:<8.5f}".format(valid_score))

Valid F1 score: 0.79691 


Model for Test Prediction with Full Trainset

In [46]:
#6 months period used for validation is 2017-07-01 till 2017-12-01
train_full_model = train.dropna(subset=['Age_lag_1'])
print(train_model.shape)

(12311, 64)


In [47]:
#Light GBM model
y_train_full = train_full_model[target]
x_train_full = train_full_model[features]
designation_cols = [col for col in train.columns if 'Designation_lag_' in col]
categoricals = designation_cols + ['Gender', 'Education_Level', 'Joining Designation']
train_set_full = lgb.Dataset(x_train_full,y_train_full , 
#                                 categorical_feature=categoricals
                       )


In [48]:
#Train full dataset using number of iterations obtained from earlier model best iteration
model_full = lgb.train(params, train_set_full, num_boost_round = valid_iteration, 
                  early_stopping_rounds = 100, 
                  keep_training_booster=True,
                  feval = lgb_f1_score,
                  valid_sets=[train_set_full], 
                  verbose_eval = 50,
                 )

Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.879984	training's F1-cutoff: 0.753344
[100]	training's auc: 0.920971	training's F1-cutoff: 0.799098
[150]	training's auc: 0.948019	training's F1-cutoff: 0.8379
[200]	training's auc: 0.963362	training's F1-cutoff: 0.865266
[250]	training's auc: 0.974778	training's F1-cutoff: 0.889369
[300]	training's auc: 0.982629	training's F1-cutoff: 0.909591
Did not meet early stopping. Best iteration is:
[335]	training's auc: 0.986655	training's F1-cutoff: 0.922252


In [49]:
#Predict train labels to compute the cutoff for test label conversion from probabilities
iteration_full = model_full.best_iteration
train_full_preds =  model_full.predict(x_train_full, num_iteration=iteration_full)
opt_cutoff_train_full, f1score = get_opt_cutoff_prec(y_train_full,train_full_preds)

In [50]:
#predict test set using the model 
predictions= model_full.predict(test[features], num_iteration=iteration_full)

#use full train probability threshold for converting test probability into test labels
test_labels = convert_probtolabels(predictions,cutoff=opt_cutoff_train_full)  

In [51]:
test_labels.shape

(741,)

In [52]:
subm['Target']=test_labels
subm.to_csv('submission.csv',index=False)

In [53]:
pd.read_csv('submission.csv').head()

,Emp_ID,Target
0,394,0
1,173,0
2,1090,0
3,840,0
4,308,0
